Import the repository from GitHub.

In [1]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/DIQ_Project2024.git

Cloning into 'DIQ_Project2024'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 32 (delta 12), reused 10 (delta 1), pack-reused 0 (from 0)
Unpacking objects: 100% (32/32), 90.79 KiB | 2.75 MiB/s, done.


In [2]:
%cd /kaggle/working/DIQ_Project2024

/kaggle/working/DIQ_Project2024


Import the libraries:

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

Import the data:

In [4]:
SERVICES = pd.read_csv('/kaggle/working/DIQ_Project2024/data/raw/Comune-di-Milano-Servizi-alla-persona-parrucchieri-estetisti.csv',sep=';',encoding='unicode_escape')
SERVICES.head()

,Tipo esercizio pa,Ubicazione,Tipo via,Via,Civico,Codice via,ZD,Prevalente,Superficie altri usi,Superficie lavorativa
0,NaN,LGO DEI GELSOMINI N. 10 (z.d. 6),LGO,DEI GELSOMINI,10,5394.0,6,NaN,NaN,55.0
1,NaN,PZA FIDIA N. 3 (z.d. 9),PZA,FIDIA,3,1144.0,9,CENTRO MASSAGGI RILASSANTI NON ESTETICI,2.0,28.0
2,NaN,VIA ADIGE N. 10 (z.d. 5),VIA,ADIGE,10,4216.0,5,CENTRO BENESSERE,2.0,27.0
3,NaN,VIA BARACCHINI FLAVIO N. 9 (z.d. 1),VIA,BARACCHINI FLAVIO,9,356.0,1,TRUCCO SEMIPERMANENTE,NaN,NaN
4,NaN,VIA BERGAMO N. 12 (z.d. 4),VIA,BERGAMO,12,3189.0,4,NaN,NaN,50.0


Basic operations to inspect data:

In [5]:
SERVICES.shape

(3909, 10)

In [6]:
SERVICES.dtypes

Tipo esercizio pa         object
Ubicazione                object
Tipo via                  object
Via                       object
Civico                    object
Codice via               float64
ZD                        object
Prevalente                object
Superficie altri usi     float64
Superficie lavorativa    float64
dtype: object

# 1. Duplication

In [7]:
SERVICES.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
3904    False
3905    False
3906    False
3907    False
3908    False
Length: 3909, dtype: bool

In [8]:
SERVICES.duplicated().any()

True

# 2. Completeness

In [9]:
SERVICES.isnull()

,Tipo esercizio pa,Ubicazione,Tipo via,Via,Civico,Codice via,ZD,Prevalente,Superficie altri usi,Superficie lavorativa
0,True,False,False,False,False,False,False,True,True,False
1,True,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,True,True
4,True,False,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...
3904,False,False,False,False,False,False,False,True,True,True
3905,False,False,False,False,False,False,False,True,True,False
3906,False,False,False,False,False,False,False,True,True,True
3907,False,False,False,False,True,False,False,True,True,True


In [10]:
#display the number of not null values for each column
SERVICES.count()

Tipo esercizio pa        3878
Ubicazione               3909
Tipo via                 3908
Via                      3908
Civico                   3832
Codice via               3908
ZD                       3908
Prevalente                294
Superficie altri usi      745
Superficie lavorativa    2601
dtype: int64

In [11]:
#total number of not null values
not_null_values = SERVICES.count().sum()
not_null_values

30891

In [12]:
#display the number of null values for each column
null_values = SERVICES.isnull().sum()
null_values

Tipo esercizio pa          31
Ubicazione                  0
Tipo via                    1
Via                         1
Civico                     77
Codice via                  1
ZD                          1
Prevalente               3615
Superficie altri usi     3164
Superficie lavorativa    1308
dtype: int64

In [13]:
#total number of null values
total_number_of_null = null_values.sum()
total_number_of_null

8199

In [14]:
#total number of cells
n_cells = SERVICES.shape[0] * SERVICES.shape[1]
n_cells

39090

COMPLETENESS EVALUATION:

In [15]:
completeness = (n_cells - total_number_of_null) / n_cells
# completeness = not_null_values / n_cells
completeness

0.7902532617037605

# 3. Accuracy
(To complete with external dataset maybe)

# Column Analysis

## a. Tipo esercizio pa

In [16]:
SERVICES["Tipo esercizio pa"].unique()

array([nan, 'Acconciatore', ' (z.d. 9)', 'ACCONCIATORE',
       'ACCONCIATORE;TIPO A ESTETICA MANUALE',
       'ACCONCIATORE;TIPO A ESTETICA MANUALE;TIPO B CENTRO DI ABBRONZATURA',
       'Acconciatore;TIPO A - REG.2003', 'ACCONCIATORE;TIPO A - REG.2003',
       'Acconciatore;TIPO A - REG.2003;TIPO B CENTRO DI ABBRONZATURA',
       'ACCONCIATORE;TIPO A - REG.2003;TIPO B CENTRO DI ABBRONZATURA',
       'BARBIERE', 'Centro abbronzatura', 'Centro benessere',
       'Centro benessere;TIPO A - REG.2003', 'Centro massaggi',
       'Centro massaggi;Centro benessere',
       'esecuzione di tatuaggi e piercing',
       'Esecuzione di tatuaggi e piercing', 'Estetista',
       'Estetista in profumeria;TIPO A ESTETICA MANUALE;TIPO D ESTET.APPAR.ELETTROMECC;TIPO C TRATT.ESTETICI DIMAGRIM',
       'Estetista;Manicure;Pedicure estetico',
       'Estetista;Parrucchiere per signora',
       'Estetista;Pedicure estetico', 'Manicure;Parrucchiere per signora',
       'Parrucchiere misto', 'Parrucchiere mi

In [17]:
SERVICES["Tipo esercizio pa"].nunique()

103

In [18]:
SERVICES["Tipo esercizio pa"].value_counts()

Tipo esercizio pa
Parrucchiere per signora                           1048
ACCONCIATORE                                        586
Parrucchiere per uomo                               439
TIPO A - REG.2003                                   335
TIPO A - REG.2003;TIPO B CENTRO DI ABBRONZATURA     313
                                                   ... 
TIPO A-B-C-D;Acconciatore                             1
TIPO A-B-C-D;ACCONCIATORE                             1
TIPO A-B-C-D;Estetista in profumeria                  1
TIPO A ESTETICA MANUALE;Acconciatore                  1
Truccatore                                            1
Name: count, Length: 103, dtype: int64

## b. Ubicazione

In [19]:
SERVICES["Ubicazione"].unique()

array(['LGO DEI GELSOMINI N. 10 (z.d. 6)', 'PZA FIDIA N. 3 (z.d. 9)',
       'VIA ADIGE N. 10 (z.d. 5)', ..., 'VIA CANDOGLIA N. 2 ; (z.d. 9)',
       'VIA NIRONE num.002a; (z.d. 1)', 'VIA MADONNINA N. 15 ; (z.d. 1)'],
      dtype=object)

In [20]:
SERVICES["Ubicazione"].nunique()

3554

In [21]:
SERVICES["Ubicazione"].value_counts()

Ubicazione
PZA DE ANGELI ERNESTO N. 1 ; (z.d. 7)    4
VIA VAL DI SOLE N. 22 ; (z.d. 5)         4
VIA DEI CINQUECENTO N. 27 ; (z.d. 4)     3
VIA DELLA CAPINERA N. 5 ; (z.d. 6)       3
VIA CASTALDI PANFILO N. 42 ; (z.d. 3)    3
                                        ..
PZA IMPERATORE TITO N. 6 ; (z.d. 4)      1
PZA LIMA N. 3 ; (z.d. 3)                 1
PZA MONTI GIOSIA N. 1 ; (z.d. 7)         1
PZA RISORGIMENTO N. 4 ; (z.d. 4)         1
VIA MADONNINA N. 15 ; (z.d. 1)           1
Name: count, Length: 3554, dtype: int64

## c. Tipo via 

In [22]:
SERVICES["Tipo via"].unique()

array(['LGO', 'PZA', 'VIA', 'VLE', 'CSO', nan, 'COMO', 'PAS', 'PLE',
       'ALZ', 'BST', 'FOR', 'VIE', 'VLO', 'GLL', 'RIP', 'PTA', 'SIT'],
      dtype=object)

In [23]:
SERVICES["Tipo via"].nunique()

17

In [24]:
SERVICES["Tipo via"].value_counts()

Tipo via
VIA     2978
VLE      455
CSO      186
PZA      157
PLE       61
LGO       30
GLL       10
RIP        6
ALZ        6
FOR        5
BST        4
PTA        3
VIE        2
VLO        2
PAS        1
COMO       1
SIT        1
Name: count, dtype: int64

## d. Via

In [25]:
SERVICES["Via"].unique()

array(['DEI GELSOMINI', 'FIDIA', 'ADIGE', ..., 'DURER ALBERTO',
       'CANDOGLIA', 'NIRONE'], dtype=object)

In [26]:
SERVICES["Via"].nunique()

1370

In [27]:
SERVICES["Via"].value_counts()

Via
PADOVA                42
MONZA                 40
DELLE FORZE ARMATE    26
LODI                  26
RIPAMONTI GIUSEPPE    25
                      ..
DELLA SILA             1
DELLE ERBE             1
DELLE PRIMULE          1
DELLE STELLINE         1
NIRONE                 1
Name: count, Length: 1370, dtype: int64

## e. Civico 

In [28]:
SERVICES["Civico"].unique()

array(['10', '3', '9', '12', '13', '39', '25', '6', '20', '30', '2', '67',
       '16', '111', '36', '83', '5', '22', '23', '1', '4', '91', '8',
       '19', '38', nan, '1111', '46', '54', '65', '103', '40', '29', '32',
       '7', '17', '27', '12A', '48', '61', '47', '15', '232', '234', '57',
       '75', '34', '150', '71', '55', '21', '6A', '163', '90', '14', '53',
       '70', '119F', '26', '24', '40945', '128', '3B', '41', '64', '68',
       '35', '49', '42', '99', '52', '72', '117', '79', '96', '58', '81',
       '112', '93/1', '87', '63', '18', '123', '31', '33', '11', '242',
       '46D', '28', '80', '98', '59', '37', '43', '60', '50', '44', '246',
       '326', '407', '410', '51', '110', '130', '131', '157', '174', '62',
       '107', '73', '139', '147', '168', '213', '255', '279', '45', '77',
       '4A', '187', '89', '207', '227', '244', '56', '100', '104', '74',
       '66', '92', '108', '121', '76', '233', '249', '256', '267', '270',
       '82', '116', '236', '69', '178', 

In [29]:
SERVICES["Civico"].nunique()

235

In [30]:
SERVICES["Civico"].value_counts()

Civico
2      249
1      247
4      165
3      158
5      147
      ... 
127      1
22A      1
179      1
243      1
336      1
Name: count, Length: 235, dtype: int64

## f. Codice via

In [31]:
SERVICES["Codice via"].unique()

array([5394., 1144., 4216., ..., 6602., 1518.,  640.])

In [32]:
SERVICES["Codice via"].nunique()

1376

In [33]:
SERVICES["Codice via"].value_counts()

Codice via
2275.0    42
2274.0    40
4068.0    26
6643.0    26
4066.0    25
          ..
586.0      1
2433.0     1
713.0      1
6716.0     1
640.0      1
Name: count, Length: 1376, dtype: int64

## g. ZD

In [34]:
SERVICES["ZD"].unique()

array(['6', '9', '5', '1', '4', '2', '8', '3', nan, 'ACCONCIATORE', '7'],
      dtype=object)

In [35]:
SERVICES["ZD"].nunique()

10

In [36]:
SERVICES["ZD"].value_counts()

ZD
1               603
3               531
4               463
7               437
2               429
8               391
9               387
6               353
5               313
ACCONCIATORE      1
Name: count, dtype: int64

## h. Prevalente 

In [37]:
SERVICES["Prevalente"].unique()

array([nan, 'CENTRO MASSAGGI RILASSANTI NON ESTETICI', 'CENTRO BENESSERE',
       'TRUCCO SEMIPERMANENTE', 'ESTETICA TIPO A', 'CENTRO MASSAGGI',
       'TATUAGGI E PIERCING', 'SERVIZI DI CENTRI PER IL BENESSERE FISICO',
       'ESTETICA A', 'acconciatore', 'ACCONCIATORE',
       'SERVIZI DEI SALONI DI BARBIERE E PARRUCCHIERE', 'Acconciatore',
       'PARRUCCHIERE UOMO DONNA BARBIERE', 'PARRUCCHIERE', 'ACONCIATORE',
       'ACCONCIATORE UOMO DONNA', 'acconciatore.',
       'ACCONCIATORE E ESTETICA',
       'CENTRO BENESSERE RILASSANTE NON ESTETICO',
       'CENTRO PER IL BENESSERE FISICO',
       'CENTTRO BENESSERE (ESCLUSI GLI STABILIMENTI TERMALI)',
       'SERVIZI DEI CENTRI E STABIL. PER IL BENESSERE FISICO',
       'STUDIO OLISTICO', 'CENTRO OLISTICO', 'centro massaggi',
       'MASSAGGI RILASSANTI', 'CENTRO MASSAGI RILASSANTI NON ESTETICI',
       'CENTRO MASSAGGI TUINA', 'CENTRO MASSAGGI RILASSANTI',
       'SERVIZI DI CENTRI PER IL BENESSERE',
       "ATTIVITA' DI MASSAGGI RILAS

In [38]:
SERVICES["Prevalente"].nunique()

63

In [39]:
SERVICES["Prevalente"].value_counts()

Prevalente
ACCONCIATORE                                    78
CENTRO MASSAGGI                                 52
CENTRO MASSAGGI RILASSANTI NON ESTETICI         30
ESTETICA A                                      13
acconciatore                                    13
                                                ..
SERVIZI DI CENTRI PER IL BENESSERE               1
ATTIVITA' DI MASSAGGI RILASSANTI                 1
CENTRO MASSAGGI THAI RILASSANTI NON CURATIVI     1
centro benessere                                 1
ESTETICA TIPO A - B                              1
Name: count, Length: 63, dtype: int64

## i. Superficie altri usi

In [40]:
SERVICES["Superficie altri usi"].unique()

array([ nan,   2.,   5.,  50.,  26.,  29.,  11.,  14.,  52.,   3., 195.,
         1.,  40.,  10.,  15.,  20.,   9.,  24.,   6., 150.,   4.,  13.,
        19.,   8.,  16.,  46.,   7.,  12.,  49.,  56.,  28.,  23.,  17.,
        25.,  77.,  41.,  32.,  70.,  22.,  34.,  18.,  42.,  31.,  21.,
        35.,  30.,  38.,  33.,  74.,  57.,  95.,  80.,  27.])

In [41]:
SERVICES["Superficie altri usi"].nunique()

52

In [42]:
SERVICES["Superficie altri usi"].value_counts()

Superficie altri usi
2.0      103
3.0       93
4.0       85
6.0       53
1.0       51
5.0       42
8.0       39
10.0      35
12.0      24
11.0      21
9.0       20
15.0      19
13.0      17
14.0      16
7.0       14
16.0      12
18.0       9
17.0       8
20.0       7
25.0       7
19.0       7
26.0       6
24.0       5
21.0       4
23.0       4
32.0       3
40.0       3
28.0       3
33.0       3
46.0       2
35.0       2
34.0       2
22.0       2
70.0       2
38.0       2
30.0       2
56.0       2
31.0       2
57.0       1
80.0       1
95.0       1
74.0       1
195.0      1
42.0       1
41.0       1
77.0       1
52.0       1
49.0       1
50.0       1
29.0       1
150.0      1
27.0       1
Name: count, dtype: int64

## l. Superficie lavorativa

In [43]:
SERVICES["Superficie lavorativa"].unique()

array([ 55.,  28.,  27.,  nan,  50.,  90., 123., 176.,  35.,  45.,  20.,
        33., 100.,  19.,  85.,  56.,  40.,  36.,  47., 149.,  51.,  46.,
        66.,  32.,  68.,  34.,  25.,  54.,  70.,  57.,  18.,  30., 172.,
        81.,  53.,  37.,  59.,  26.,  39.,  16.,  69.,  38.,  42.,  23.,
        31.,  43.,  44.,  49.,  65.,  41.,  58.,  86.,  48.,  63.,  62.,
        77.,  80.,  21.,  15.,  79.,  17.,  12.,  78.,  64.,  24.,  52.,
        84.,  98.,  22.,  13.,  29.,  14., 193.,  60.,  74.,  95., 110.,
        83.,  61., 114.,  96.,  73., 220., 115., 160., 111.,  88.,  67.,
       120., 108., 105.,  99.,  75.,  76.,  87., 156.,   7., 136., 180.,
       104., 102., 118.,  89.,  97.,  93.,  94.,  71.,  11., 130.,  92.,
       124.,   8.,  72., 131.,  10., 122., 247., 113., 116.,   5.,  91.,
       522.,   4.,   9., 174., 109., 182., 157., 103., 324., 112., 158.,
       128.,   6.,   1., 150., 140., 267., 195., 230.,  82., 101., 154.,
       268., 117., 126.])

In [44]:
SERVICES["Superficie lavorativa"].nunique()

145

In [45]:
SERVICES["Superficie lavorativa"].value_counts()

Superficie lavorativa
30.0     120
40.0     100
45.0      89
25.0      88
20.0      87
        ... 
193.0      1
522.0      1
4.0        1
176.0      1
126.0      1
Name: count, Length: 145, dtype: int64

# Profiling 